In [1]:
%cd ./data

/mnt/data1/home/yoshida/my_research/data


In [2]:
#!pip3 install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.10
#torch.__version__
#torchtext.__version__

In [3]:
#初回時エラー出るので2回実行する
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
from pytorch_lightning.callbacks import ModelCheckpoint
import sys
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from sklearn.metrics import classification_report

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
#MODEL_NAME = "./tapt512_60K/"

In [4]:
# https://huggingface.co/transformers/model_doc/bert.html#<class名>
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME) #bertの学習済みモデルをtokenizerとする
bert_sc = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3) #学習済みモデルを用いた分類、num_labelで分類する種類の数
bert_sc = bert_sc.cuda() #gpuに乗せて高速化させるための関数

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [5]:
def tag_auto(name):
    #データの読み込み
    df = pd.read_csv(name + ".csv",encoding="utf-8",engine="python",usecols=[1])

    with open("./all_flaming.txt","r",encoding="UTF-8") as f:
        word_lis = [s.strip() for s in f.readlines()]
        
    #print(word_lis)
    flag = 0
    text_lis = []
    f_text_lis = []
    text_labels = []
    f_text_labels = []
    for text in df["text"]:
        for word in word_lis:
            if word in text:
                f_text_lis.append(text)
                f_text_labels.append(1)
                flag = 1
                break
        if flag :
            flag = 0
        else:
            text_lis.append(text)
            text_labels.append(0)
        
    return text_lis,text_labels, f_text_lis, f_text_labels

#plan to make this after getting data from everyone
def tag_get(name):
    #load data
    df = pd.read_csv(name + ".csv",encoding="utf-8",engine="python",usecols=[1,2])
    text_lis = []
    f_text_lis = []
    s_text_lis = []
    for text,tag in zip(df["text"],df["tag"]):
        if tag == 0:
            text_lis.append(text)
        elif tag == 1:
            f_text_lis.append(text)
        else:
            s_text_lis.append(text)
    
    return text_lis,f_text_lis,s_text_lis

In [6]:
#text_list, labels, f_text, f_labels = tag_auto("./preprocess") #危険辞書からラベルづけを自動化
text_list, f_text, s_text = tag_get("./tag_decition") #手動でタグ付けしたデータの利用(usecolは別途指定)

safety_len = len(text_list)
flaming_len = len(f_text)
spam_len = len(s_text)
labels = [0] * safety_len #tag_get ver
f_labels = [1] * flaming_len
s_labels = [2] * spam_len
print(safety_len,flaming_len,spam_len)

1064 765 171


In [7]:
#データの符号化
def Encoding(text_list, labels, max_length=128):
    dataset_for_loader = []
    for idx,text in enumerate(text_list):
        encoding = tokenizer(text, max_length=max_length,padding="max_length",truncation=True) #textを形態素解析、"pt"でtensor出力,辞書型でreturn
        encoding["labels"] = labels[idx] #add label
        encoding = {k: torch.tensor(v) for k,v in encoding.items()}  
        dataset_for_loader.append(encoding)
    return dataset_for_loader

max_length = 128
dataset_load_safe = Encoding(text_list,labels,max_length)
dataset_load_flaming = Encoding(f_text,f_labels,max_length)
dataset_load_spam = Encoding(s_text, s_labels,max_length)  #3値分類に使用、スパム

In [8]:
#確認用
print(dataset_load_safe[19])
print("-"*60)
print(dataset_load_flaming[19])
print("-"*60)
print(dataset_load_spam[19])

{'input_ids': tensor([    2, 13312,   108,    11,    23,  1746, 28487, 17364, 30365,  5098,
        17015,   205,    23,  1746, 28487,   679,  1746, 28487, 17364, 30365,
         7309,    12,  3173,     7,    58,    16,     3,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [9]:
def random_list(lis,seed=42):
    random.seed()
    random.shuffle(lis)
    return lis

#データセットの分割
def dataset_separate(dataset):
    n = len(dataset)
    n_train = int(0.6*n)
    n_val = int(0.2*n)
    dataset_train = dataset[:n_train]
    dataset_val = dataset[n_train:n_train + n_val]
    dataset_test = dataset[n_train+n_val:]
    return dataset_train, dataset_val, dataset_test

t_dataset_train,t_dataset_val, t_dataset_test = dataset_separate(dataset_load_safe)
f_dataset_train,f_dataset_val, f_dataset_test = dataset_separate(dataset_load_flaming)
s_dataset_train,s_dataset_val, s_dataset_test = dataset_separate(dataset_load_spam) #spam ver
dataset_train = t_dataset_train + f_dataset_train + s_dataset_train
dataset_val = t_dataset_val + f_dataset_val + s_dataset_val
dataset_test = t_dataset_test + f_dataset_test + s_dataset_test
dataset_train = random_list(dataset_train)
dataset_val = random_list(dataset_val)
dataset_test = random_list(dataset_test)

#データセットからデータローダを作成
#学習データはshuffle=Trueにする
dataloader_train = DataLoader(dataset_train, batch_size=128, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [10]:
class BertForSequenceClassification_pl(pl.LightningModule):
    
    def __init__(self, model_name, num_labels, lr):
        #model_name: name of transformers model
        #num_labels: num of labels
        #lr: learning rate
        
        super().__init__()
        #num_labels,lrを保存
        self.save_hyperparameters()
        
        #BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
    
    #テストデータのミニバッチが与えられたときテストデータを評価する指標を計算する関数を書く
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log("train_loss", loss,prog_bar=True,logger=True,on_epoch=True,on_step=True) #損失をtrain_lossの名前でログを取る
        return loss
    
    #検証データ版の評価関数
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log("val_loss", val_loss,prog_bar=True,logger=True,on_epoch=True,on_step=True)
        
    def test_step(self, batch, batch_idx):
        labels = batch.pop("labels") #バッチからラベルの取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = (labels_predicted == labels).sum().item()
        accuracy = num_correct / labels.size(0) #精度
        self.log("accuracy", accuracy)
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr = self.hparams.lr)

In [11]:
checkpoint = ModelCheckpoint(
    monitor="val_loss", mode="min", save_top_k=1,save_weights_only=True, dirpath="model/")

#学習方法の指定
trainer = pl.Trainer(gpus=1, max_epochs=100, callbacks = [checkpoint],logger=[pl_loggers.TensorBoardLogger("logs/")])

/mnt/data1/home/yoshida/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory model/ exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [12]:
model = BertForSequenceClassification_pl(MODEL_NAME, num_labels=3, lr=1e-5)

#fine-Tuning
trainer.fit(model, dataloader_train, dataloader_val)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Validation sanity check: 0it [00:00, ?it/s]

/mnt/data1/home/yoshida/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [13]:
best_model_path = checkpoint.best_model_path #file of best model
print("ベストモデルのファイル: ", checkpoint.best_model_path)
print("ベストモデルの検証データに対する損失: ", checkpoint.best_model_score)

ベストモデルのファイル:  /mnt/data1/home/yoshida/my_research/data/model/epoch=6-step=69-v3.ckpt
ベストモデルの検証データに対する損失:  tensor(0.5564, device='cuda:0')


In [14]:
#%load_ext tensorboard    #sshで6006につなぐと見られるよ
#%tensorboard --logdir ./logs

ModuleNotFoundError: No module named 'tensorboard    #sshで6006につなぐと見られるよ'

In [ ]:
test = trainer.test(test_dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

In [ ]:
print(test)

In [ ]:
#load to pytorch-lightning model
model = BertForSequenceClassification_pl.load_from_checkpoint(best_model_path)

#preserve at "./model_transformers" involving transformers model
model.bert_sc.save_pretrained("./model_tapt_epo40_128")

In [ ]:
bert_sc = BertForSequenceClassification.from_pretrained("./model_ex",num_labels=2)
bert_sc = bert_sc.cuda()